<a href="https://colab.research.google.com/github/ucancallmetigger-ui/Image_proccesing_Catsa_and_Dogs/blob/main/dogs_and_cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install datasets torchvision torch --quiet

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from datasets import load_dataset
from PIL import Image
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = load_dataset("microsoft/cats_vs_dogs")['train']

class HFImageDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.hf_dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        item = self.hf_dataset[idx]
        image = item['image'].convert("RGB")
        label = item['labels']
        if self.transform:
            image = self.transform(image)
        return image, label

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

full_dataset = HFImageDataset(dataset, transform=transform_train)

cat_indices = [i for i in range(len(dataset)) if dataset[i]['labels'] == 0]
dog_indices = [i for i in range(len(dataset)) if dataset[i]['labels'] == 1]

random.seed(42)
train_cat = random.sample(cat_indices, 500)
train_dog = random.sample(dog_indices, 500)
train_indices = train_cat + train_dog
random.shuffle(train_indices)

remaining_cat = list(set(cat_indices) - set(train_cat))
remaining_dog = list(set(dog_indices) - set(train_dog))
test_cat = random.sample(remaining_cat, 500)
test_dog = random.sample(remaining_dog, 500)
test_indices = test_cat + test_dog
random.shuffle(test_indices)

train_dataset = torch.utils.data.Subset(full_dataset, train_indices)

test_dataset_raw = HFImageDataset(dataset, transform=transform_test)
test_dataset = torch.utils.data.Subset(test_dataset_raw, test_indices)

trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
testloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, 1)

model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

for epoch in range(25):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.float().unsqueeze(1).to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    scheduler.step()

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.float().unsqueeze(1).to(device)
        outputs = model(inputs)
        predicted = torch.sigmoid(outputs) > 0.5
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"دقت نهایی روی ۱۰۰۰ تصویر تست: {accuracy:.2f}%")